# Pandas + duckdb

## load sakila sqlite -> duckdb

In [1]:
import duckdb

with open("sql/load_sakila.sql", "r") as sql_file:
    loading_script = sql_file.read()

loading_script

"INSTALL sqlite;\n\nLOAD sqlite;\n\nCALL sqlite_attach ('data/sqlite-sakila.db');"